# Domain Adaptation Via Activation Shaping

**Teacher assistant** Iurada Leonardo

**Students**

- Bar Giorgio
- Distefano Giuseppe
- Incaviglia Salvatore

## Code

### Libraries

In [ ]:
!pip install torch torchvision tqdm torchmetrics

import torch

import torch.backends.mps
from argparse import ArgumentParser

import os
import torchvision.transforms as T

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10

import numpy as np
import random
from PIL import Image

import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

import torch.nn.functional as F
from torchmetrics import Accuracy
from tqdm import tqdm

import logging
import warnings

import argparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 5.4 MB/s eta 0:00:00


### Setup PACS dataset and environment

In [ ]:
# Download PACS Dataset Images and Labels
!git clone https://github.com/MachineLearning2020/Homework3-PACS/
!git clone https://github.com/silvia1993/DANN_Template/

# Setup data
!rm -rf data || true
!rm -rf record || true
!mkdir data
!mkdir data/kfold
!cp -r Homework3-PACS/PACS/ data/kfold
!cp DANN_Template/txt_lists/*.txt data
!rm -rf Homework3-PACS/
!rm -rf DANN_Template/
!rm -rf sample_data || true

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032
Receiving objects: 100% (10032/10032), 174.13 MiB | 27.68 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (9993/9993), done.
Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Receiving objects: 100% (23/23), 33.86 KiB | 1.30 MiB/s, done.
Resolving deltas: 100% (5/5), done.


### Globals

#### globals

In [ ]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

CONFIG = dotdict({})

if torch.cuda.is_available():
    CONFIG.device = 'cuda'
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    CONFIG.device = 'mps'
else:
    CONFIG.device = 'cpu'

CONFIG.dtype = torch.float32

#### parse args

In [ ]:
def _clear_args(parsed_args):
    parsed_args.experiment_args = eval(parsed_args.experiment_args)
    parsed_args.dataset_args = eval(parsed_args.dataset_args)
    return parsed_args


def parse_arguments():
    parser = ArgumentParser()

    parser.add_argument('--seed', type=int, default=0, help='Seed used for deterministic behavior')
    parser.add_argument('--test_only', action='store_true', help='Whether to skip training')
    parser.add_argument('--cpu', action='store_true', help='Whether to force the usage of CPU')

    parser.add_argument('--experiment', type=str, default='baseline')
    parser.add_argument('--experiment_name', type=str, default='baseline')
    parser.add_argument('--experiment_args', type=str, default='{}')
    parser.add_argument('--dataset_args', type=str, default='{}')

    parser.add_argument('--batch_size', type=int, default=128)
    parser.add_argument('--epochs', type=int, default=30)
    parser.add_argument('--num_workers', type=int, default=5)
    parser.add_argument('--grad_accum_steps', type=int, default=1)

    return _clear_args(parser.parse_args())

#### Global variables

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3               # The initial Learning Rate
MOMENTUM = 0.9          # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5     # Regularization, you can keep this at the default
NUM_EPOCHS = 30         # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20          # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1             # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

### Dataset

#### utils

In [ ]:
class BaseDataset(Dataset):
    def __init__(self, examples, transform):
        self.examples = examples
        self.T = transform


    def __len__(self):
        return len(self.examples)


    def __getitem__(self, index):
        x, y = self.examples[index]
        x = Image.open(x).convert('RGB')
        x = self.T(x).to(CONFIG.dtype)
        y = torch.tensor(y).long()
        return x, y

######################################################

class DomainAdaptationDataset(Dataset):
    def __init__(self, source_examples, target_examples, transform):
        self.source_examples = source_examples
        self.target_examples = target_examples
        self.T = transform


    def __len__(self):
        return len(self.source_examples)


    def __getitem__(self, index):
        src_x, src_y = self.source_examples[index]
        src_x = Image.open(src_x).convert('RGB')
        src_x = self.T(src_x).to(CONFIG.dtype)
        src_y = torch.tensor(src_y).long()

        # randomly sample 'target_examples' to obtain targ_x
        target_idx = torch.randint(len(self.target_examples), size=(1,)).item()
        targ_x, _ = self.target_examples[target_idx]
        targ_x = Image.open(targ_x).convert('RGB')
        targ_x = self.T(targ_x).to(CONFIG.dtype)

        return src_x, src_y, targ_x

######################################################

# [OPTIONAL] TODO: modify 'BaseDataset' for the Domain Generalization setting.
# Hint: combine the examples from the 3 source domains into a single 'examples' list
#class DomainGeneralizationDataset(Dataset):
#    def __init__(self, examples, transform):
#        self.examples = examples
#        self.T = transform
#
#    def __len__(self):
#        return len(self.examples)
#
#    def __getitem__(self, index):
#        x1, x2, x3 = self.examples[index]
#        x1, x2, x3 = self.T(x1), self.T(x2), self.T(x3)
#        targ_x = self.T(targ_x)
#        return x1, x2, x3

######################################################

class SeededDataLoader(DataLoader):
    def __init__(self, dataset: Dataset, batch_size=1, shuffle=None,
                 sampler=None,
                 batch_sampler=None,
                 num_workers=0, collate_fn=None,
                 pin_memory=False, drop_last=False, timeout=0,
                 worker_init_fn=None, multiprocessing_context=None,
                 generator=None, *, prefetch_factor=None, persistent_workers=False,
                 pin_memory_device=""):

        if not CONFIG.use_nondeterministic:
            def seed_worker(worker_id):
                worker_seed = torch.initial_seed() % 2**32
                np.random.seed(worker_seed)
                random.seed(worker_seed)

            generator = torch.Generator()
            generator.manual_seed(CONFIG.seed)

            worker_init_fn = seed_worker

        super().__init__(dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn,
                         pin_memory, drop_last, timeout, worker_init_fn, multiprocessing_context, generator,
                         prefetch_factor=prefetch_factor, persistent_workers=persistent_workers,
                         pin_memory_device=pin_memory_device)

#### PACS

In [ ]:
def get_transform(size, mean, std, preprocess):
    transform = []
    if preprocess:
        transform.append(T.Resize(256))
        transform.append(T.RandomResizedCrop(size=size, scale=(0.7, 1.0)))
        transform.append(T.RandomHorizontalFlip())
    else:
        transform.append(T.Resize(size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean, std))
    return T.Compose(transform)


def load_data():
    CONFIG.num_classes = 7
    CONFIG.data_input_size = (3, 224, 224)

    # Create transforms
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) # ImageNet Pretrain statistics
    train_transform = get_transform(size=224, mean=mean, std=std, preprocess=True)
    test_transform = get_transform(size=224, mean=mean, std=std, preprocess=False)

    # Load examples
    source_examples, target_examples = [], []
    if CONFIG.experiment in ['baseline', 'random', 'domain_adaptation']:
        # Load source
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['source_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            source_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

        # Load target
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['target_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            target_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

    # create Dataset object for the baseline and the Random Maps Ablation experiments
    if CONFIG.experiment in ['baseline', 'random']:
        train_dataset = BaseDataset(source_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    # create Dataset object for the Domain Adaptation experiments
    if CONFIG.experiment in ['domain_adaptation']:
        train_dataset = DomainAdaptationDataset(source_examples, target_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    ######################################################

    # Dataloaders
    train_loader = SeededDataLoader(
        train_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=True,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    test_loader = SeededDataLoader(
        test_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    return {'train': train_loader, 'test': test_loader}

### Models

#### Base ResNet 18

In [ ]:
class BaseResNet18(nn.Module):
    def __init__(self):
        super(BaseResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)


    def forward(self, x):
        return self.resnet(x)

#### ASH ResNet 18

In [ ]:
def get_activation_shaping_hook (mask):

    # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
    # The hook captures mask variable from the parent scope, to update it,
    # remove() the hook and register a new one with the updated mask.

    def activation_shaping_hook(module, input, output):

        # print('ASH module registered on: ', module, 'mask shape: ', mask.shape, 'mask sum(): ', mask.sum())

        # binarize both activation map and mask using zero as threshold
        A_binary = torch.where(output<=0, torch.tensor(0.0), torch.tensor(1.0))
        M_binary = torch.where(mask<=0, torch.tensor(0.0), torch.tensor(1.0))

        # return the element-wise product of activation map and mask
        shaped_output = A_binary * M_binary
        return shaped_output

    return activation_shaping_hook

######################################################

# 'BaseResNet18' including the Activation Shaping Module
class ASHResNet18(nn.Module):
    def __init__(self):
        super(ASHResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)

        self.hook_handles = {}


    def register_activation_shaping_hook(self, layer_name, mask):
        hook = get_activation_shaping_hook(mask)

        for name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and name == layer_name):
                if not name in self.hook_handles:
                    print('Insert activation shaping layer after ', name, module)
                    self.hook_handles[name] = module.register_forward_hook(hook)


    def remove_activation_shaping_hook(self, name):
        if name in self.hook_handles and self.hook_handles[name] is not None:
            handle = self.hook_handles[name]
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

#### ASH ResNet 18 for Domain Adaptation

In [ ]:
# 'BaseResNet18' including the Activation Shaping Module for Domain Adaptation
class ASHResNet18DomainAdaptation(nn.Module):
    def __init__(self, module_placement=None):
        super(ASHResNet18DomainAdaptation, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)
        '''
        self.module_placement = module_placement
        if module_placement==[]:
            self.module_placement = None
        '''
        self.activation_maps = {}
        self.activation_map_hook_handles = {}
        self.activation_shaping_hook_handles = {}


    def get_extract_activation_map_hook (self, layer_name):
        def extract_activation_map_hook(module, input, output):
            self.activation_maps[layer_name] = output.detach().clone()
        return extract_activation_map_hook


    def get_activation_shaping_hook (self, mask):
        # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
        def activation_shaping_hook (module, input, output):
            A_binary = torch.where(output <= 0, torch.tensor(0.0), torch.tensor(1.0))
            M_binary = torch.where(mask <= 0, torch.tensor(0.0), torch.tensor(1.0))
            shaped_output = A_binary* M_binary
            return shaped_output
        return activation_shaping_hook


    def register_extract_activation_map_hooks (self, module_placement):
        # Register hook(s) (1st hook) to store activation map
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                hook = self.get_extract_activation_map_hook(layer_name)
                self.activation_map_hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_extract_activation_map_hooks (self):
        # Remove hook(s) used to store activation map
        for layer_name, handle in self.activation_map_hook_handles.items():
            handle.remove()


    def register_activation_shaping_hooks (self, module_placement):
        # Register the Activation Shaping Module hook(s) (2nd hook)
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                hook = self.get_activation_shaping_hook(self.activation_maps[layer_name])
                self.activation_shaping_hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_activation_shaping_hooks (self):
        # Remove hook(s) used to perform Activation Shaping
        for layer_name, handle in self.activation_shaping_hook_handles.items():
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

### Run

#### Evaluate

In [ ]:
@torch.no_grad()
def evaluate(model, data):
    model.eval()

    acc_meter = Accuracy(task='multiclass', num_classes=CONFIG.num_classes)
    acc_meter = acc_meter.to(CONFIG.device)

    loss = [0.0, 0]
    for x, y in tqdm(data):
        with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
            x, y = x.to(CONFIG.device), y.to(CONFIG.device)
            logits = model(x)
            acc_meter.update(logits, y)
            loss[0] += F.cross_entropy(logits, y).item()
            loss[1] += x.size(0)

    accuracy = acc_meter.compute()
    loss = loss[0] / loss[1]
    logging.info(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss}')
    print(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss} \n')

#### Train

In [ ]:
def train(model, data):
    # Create optimizers & schedulers
    optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.0005, momentum=0.9, nesterov=True, lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(CONFIG.epochs * 0.8), gamma=0.1)
    scaler = torch.cuda.amp.GradScaler(enabled=True)

    # Load checkpoint (if it exists)
    cur_epoch = 0
    if os.path.exists(os.path.join('record', CONFIG.experiment_name, 'last.pth')):
        checkpoint = torch.load(os.path.join('record', CONFIG.experiment_name, 'last.pth'))
        cur_epoch = checkpoint['epoch']
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        model.load_state_dict(checkpoint['model'])

    # Optimization loop
    for epoch in range(cur_epoch, CONFIG.epochs):
        model.train()

        for batch_idx, batch in enumerate(tqdm(data['train'])):

            # use a separate invocation of autocast for every forward pass.
            if CONFIG.experiment in ['domain_adaptation']:
                with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
                    model.eval()
                    # Register hook(s) to store the target domain activation maps and perform a forward pass
                    src_x, src_y, targ_x = batch
                    targ_x = targ_x.to(CONFIG.device)
                    module_placement = CONFIG.experiment_args['module_placement']
                    model.register_extract_activation_map_hooks(module_placement)
                    with torch.no_grad():
                        model(targ_x)
                    model.remove_extract_activation_map_hooks()
                    model.train()

            # Compute loss
            with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):

                if CONFIG.experiment in ['baseline']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)
                elif CONFIG.experiment in ['random']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)
                elif CONFIG.experiment in ['domain_adaptation']:
                    src_x, src_y, targ_x = batch
                    src_x, src_y = src_x.to(CONFIG.device), src_y.to(CONFIG.device)
                    # Register the Activation Shaping hook(s) and perform a forward pass (source domain images)
                    model.register_activation_shaping_hooks(module_placement)
                    loss = F.cross_entropy(model(src_x), src_y)
                    model.remove_activation_shaping_hooks()

            # Optimization step
            scaler.scale(loss / CONFIG.grad_accum_steps).backward()

            if ((batch_idx + 1) % CONFIG.grad_accum_steps == 0) or (batch_idx + 1 == len(data['train'])):
                scaler.step(optimizer)
                optimizer.zero_grad(set_to_none=True)
                scaler.update()

        scheduler.step()

        # Test current epoch
        logging.info(f'[TEST @ Epoch={epoch}]')
        evaluate(model, data['test'])

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'model': model.state_dict()
        }
        torch.save(checkpoint, os.path.join('record', CONFIG.experiment_name, 'last.pth'))

#### Main

In [ ]:
layer_output_shapes = {
    'layer1.0.conv1': (64, 56, 56),         'layer1.0.relu': (64, 56, 56),          'layer1.0.conv2': (64, 56, 56),
    'layer1.1.conv1': (64, 56, 56),         'layer1.1.bn1': (64, 56, 56),           'layer1.1.relu': (64, 56, 56),          'layer1.1.conv2': (64, 56, 56),         'layer1.1.bn2': (64, 56, 56),
    'layer2.0.conv1': (128, 28, 28),        'layer2.0.relu': (128, 28, 28),         'layer2.0.conv2': (128, 28, 28),
    'layer2.1.conv1': (128, 28, 28),        'layer2.1.relu': (128, 28, 28),         'layer2.1.conv2': (128, 28, 28),
    'layer3.0.conv1': (256, 14, 14),        'layer3.0.relu': (256, 14, 14),         'layer3.0.conv2': (256, 14, 14),
    'layer3.1.conv1': (256, 14, 14),        'layer3.1.relu': (256, 14, 14),         'layer3.1.conv2': (256, 14, 14),
    'layer4.0.conv1': (512, 7, 7),          'layer4.0.relu': (512, 7, 7),           'layer4.0.conv2': (512, 7, 7),
    'layer4.1.conv1': (512, 7, 7),          'layer4.1.relu': (512, 7, 7),           'layer4.1.conv2': (512, 7, 7)
}


def create_random_mask(layer_name = 'layer4.1.conv2', mask_out_ratio = 0.0):
    # create a mask tensor with a given ratio of zeros
    print('mask_out_ratio: ',mask_out_ratio)
    layer_output_shape = layer_output_shapes[layer_name]

    rand_mat = torch.rand(layer_output_shape).to(CONFIG.device)
    mask = torch.where(rand_mat <= mask_out_ratio, 0.0, 1.0).to(CONFIG.device)
    return mask


def main ():
    # Load dataset
    data = load_data()

    # Load model
    if CONFIG.experiment in ['baseline']:
        model = BaseResNet18()

    ######################################################

    elif CONFIG.experiment in ['random']:
        model = ASHResNet18()
        module_placement = CONFIG.experiment_args['module_placement']
        mask_out_ratio = CONFIG.experiment_args['mask_out_ratio']

        for layer_name in module_placement:
            random_mask = create_random_mask(layer_name = layer_name, mask_out_ratio = mask_out_ratio)
            print('layer_name', layer_name, 'random_mask sum(): ', random_mask.sum())
            model.register_activation_shaping_hook(layer_name, random_mask)

    elif CONFIG.experiment in ['domain_adaptation']:
         module_placement = CONFIG.experiment_args['module_placement']
         model= ASHResNet18DomainAdaptation(module_placement)

    ######################################################

    model.to(CONFIG.device)

    if not CONFIG.test_only:
        train(model, data)
    else:
        evaluate(model, data['test'])


## 0 - Reproduce the Baseline

### 0.1 - Baseline

#### 0.1.1 - Art painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': '0_1_1',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 0.1.2 - Art painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': '0_1_2',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 0.1.3 - Art painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': '0_1_3',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

## 1 - Activation Shaping module

### 1.1 - ASH module after last convolutional layer

#### 1.1.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_1_1',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.2 - Art Painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_1_2',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.3 - Art Painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_1_3',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

### 1.2 - ASH module after all convolutional layers

#### 1.2.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_2_1',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.2.2 - Art Painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_2_2',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.2.3 - Art Painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_2_3',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

### 1.3 - ASH module after every three convolutional layers

#### 1.3.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_3_1',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.3.2 - Art Painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_3_2',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.3.3 - Art Painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '1_3_3',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

## 2 - Random Activation Maps

### 2.1 - ASH module after last convolutional layer

#### 2.1.1 - Art Painting &rightarrow; Cartoon, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_1',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.2 - Art Painting &rightarrow; Sketch, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_2',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.3 - Art Painting &rightarrow; Photo, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_3',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.4 - Art Painting &rightarrow; Cartoon, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_4',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.5 - Art Painting &rightarrow; Sketch, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_5',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.6 - Art Painting &rightarrow; Photo, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_6',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.7 - Art Painting &rightarrow; Cartoon, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_7',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.8 - Art Painting &rightarrow; Sketch, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_8',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.1.9 - Art Painting &rightarrow; Photo, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_1_9',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

### 2.2 - ASH module after every convolutional layer

#### 2.2.1 - Art Painting &rightarrow; Cartoon, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_1',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.2 - Art Painting &rightarrow; Sketch, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_2',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.3 - Art Painting &rightarrow; Photo, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_3',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.4 - Art Painting &rightarrow; Cartoon, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_4',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.5 - Art Painting &rightarrow; Sketch, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_5',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.6 - Art Painting &rightarrow; Photo, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_6',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.7 - Art Painting &rightarrow; Cartoon, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_7',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.8 - Art Painting &rightarrow; Sketch, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_8',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 2.2.9 - Art Painting &rightarrow; Photo, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_2_9',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

### 2.3 - ASH module after every three convolutional layers

#### 2.3.1 - Art Painting &rightarrow; Cartoon, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_1',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_1', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 88.8MB/s]


mask_out_ratio:  0.4
layer_name layer1.0.conv1 random_mask sum():  tensor(120682., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.4
layer_name layer1.1.conv2 random_mask sum():  tensor(120163., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.4
layer_name layer2.1.conv1 random_mask sum():  tensor(60275., device='cuda:0')
Insert activation shaping layer after  layer2.1.conv1 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.4
layer_name layer3.0.conv2 random_mask sum():  tensor(30148., device='cuda:0')
Insert activation shaping layer after  layer3.0.conv2 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.4
layer_name layer4.0.conv1 random_mask sum():  ten

100%|██████████| 19/19 [00:09<00:00,  1.91it/s]


Accuracy: 13.57 - Loss: 0.01614898153335975 



...

Accuracy: 27.94 - Loss: 0.015257839355045618 



100%|██████████| 19/19 [00:07<00:00,  2.70it/s]


Accuracy: 27.69 - Loss: 0.015244867411082922 



100%|██████████| 19/19 [00:06<00:00,  3.00it/s]


Accuracy: 26.83 - Loss: 0.015314963897340534 



#### 2.3.2 - Art Painting &rightarrow; Sketch, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_2',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_2', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.4
layer_name layer1.0.conv1 random_mask sum():  tensor(120682., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.4
layer_name layer1.1.conv2 random_mask sum():  tensor(120163., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.4
layer_name layer2.1.conv1 random_mask sum():  tensor(60275.,

100%|██████████| 31/31 [00:12<00:00,  2.53it/s]


Accuracy: 16.19 - Loss: 0.017017997594606666 



100%|██████████| 31/31 [00:11<00:00,  2.79it/s]


Accuracy: 11.81 - Loss: 0.016273516709339107 



...

Accuracy: 21.00 - Loss: 0.015319876526414847 



100%|██████████| 31/31 [00:11<00:00,  2.81it/s]


Accuracy: 21.68 - Loss: 0.015393207442763502 



100%|██████████| 31/31 [00:11<00:00,  2.78it/s]


Accuracy: 21.46 - Loss: 0.015367817350791957 



#### 2.3.3 - Art Painting &rightarrow; Photo, mor = 0.4

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_3',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_3', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.4
layer_name layer1.0.conv1 random_mask sum():  tensor(120682., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.4
layer_name layer1.1.conv2 random_mask sum():  tensor(120163., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.4
layer_name layer2.1.conv1 random_mask sum():  tensor(60275., 

100%|██████████| 14/14 [00:05<00:00,  2.67it/s]


Accuracy: 16.77 - Loss: 0.016646911712463745 



100%|██████████| 14/14 [00:05<00:00,  2.72it/s]


Accuracy: 27.84 - Loss: 0.015544647727897782 



...

Accuracy: 52.28 - Loss: 0.011256007234493416 



100%|██████████| 14/14 [00:05<00:00,  2.64it/s]


Accuracy: 54.13 - Loss: 0.011229364029661624 



100%|██████████| 14/14 [00:05<00:00,  2.65it/s]


Accuracy: 53.47 - Loss: 0.011203410025842176 



#### 2.3.4 - Art Painting &rightarrow; Cartoon, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_4',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_4', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
layer_name layer1.0.conv1 random_mask sum():  tensor(90359., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.55
layer_name layer1.1.conv2 random_mask sum():  tensor(89968., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.55
layer_name layer2.1.conv1 random_mask sum():  tensor(4485

100%|██████████| 19/19 [00:07<00:00,  2.60it/s]


Accuracy: 14.76 - Loss: 0.015993106588975563 



100%|██████████| 19/19 [00:05<00:00,  3.24it/s]


Accuracy: 15.49 - Loss: 0.015883924204334873 



...

Accuracy: 24.66 - Loss: 0.015466206106309598 



100%|██████████| 19/19 [00:05<00:00,  3.29it/s]


Accuracy: 24.74 - Loss: 0.015500482235345418 



100%|██████████| 19/19 [00:05<00:00,  3.30it/s]


Accuracy: 25.73 - Loss: 0.015301645819237614 



#### 2.3.5 - Art Painting &rightarrow; Sketch, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_5',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_5', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
layer_name layer1.0.conv1 random_mask sum():  tensor(90359., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.55
layer_name layer1.1.conv2 random_mask sum():  tensor(89968., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.55
layer_name layer2.1.conv1 random_mask sum():  tensor(44856

100%|██████████| 31/31 [00:12<00:00,  2.55it/s]


Accuracy: 11.10 - Loss: 0.016890399114385942 



100%|██████████| 31/31 [00:11<00:00,  2.72it/s]


Accuracy: 10.44 - Loss: 0.01645401950767247 



...

Accuracy: 22.19 - Loss: 0.015362219277700113 



100%|██████████| 31/31 [00:10<00:00,  2.84it/s]


Accuracy: 22.47 - Loss: 0.015446710113230235 



100%|██████████| 31/31 [00:11<00:00,  2.79it/s]


Accuracy: 22.42 - Loss: 0.015302868161258518 



#### 2.3.6 - Art Painting &rightarrow; Photo, mor = 0.55

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_6',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_6', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
layer_name layer1.0.conv1 random_mask sum():  tensor(90359., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.55
layer_name layer1.1.conv2 random_mask sum():  tensor(89968., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.55
layer_name layer2.1.conv1 random_mask sum():  tensor(44856.

100%|██████████| 14/14 [00:05<00:00,  2.53it/s]


Accuracy: 18.68 - Loss: 0.016169988537976843 



100%|██████████| 14/14 [00:05<00:00,  2.46it/s]


Accuracy: 23.35 - Loss: 0.015840710851246727 



...

Accuracy: 42.51 - Loss: 0.013182342552139374 



100%|██████████| 14/14 [00:05<00:00,  2.62it/s]


Accuracy: 42.10 - Loss: 0.013247511986486927 



100%|██████████| 14/14 [00:05<00:00,  2.73it/s]


Accuracy: 43.17 - Loss: 0.013199923852246678 



#### 2.3.7 - Art Painting &rightarrow; Cartoon, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_7',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_7', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
layer_name layer1.0.conv1 random_mask sum():  tensor(50272., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.75
layer_name layer1.1.conv2 random_mask sum():  tensor(50075., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.75
layer_name layer2.1.conv1 random_mask sum():  tensor(2501

100%|██████████| 19/19 [00:06<00:00,  2.89it/s]


Accuracy: 15.36 - Loss: 0.015959075032240703 



100%|██████████| 19/19 [00:06<00:00,  3.11it/s]


Accuracy: 13.65 - Loss: 0.015832930955870565 



...

Accuracy: 19.71 - Loss: 0.0155666461881924 



100%|██████████| 19/19 [00:05<00:00,  3.18it/s]


Accuracy: 20.05 - Loss: 0.015576451457401184 



100%|██████████| 19/19 [00:06<00:00,  3.07it/s]


Accuracy: 20.35 - Loss: 0.01554930169427761 



#### 2.3.8 - Art Painting &rightarrow; Sketch, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_8',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_8', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
layer_name layer1.0.conv1 random_mask sum():  tensor(50272., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.75
layer_name layer1.1.conv2 random_mask sum():  tensor(50075., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.75
layer_name layer2.1.conv1 random_mask sum():  tensor(25010

100%|██████████| 31/31 [00:11<00:00,  2.61it/s]


Accuracy: 7.71 - Loss: 0.016190448344952278 



100%|██████████| 31/31 [00:11<00:00,  2.80it/s]


Accuracy: 5.12 - Loss: 0.016675607246064934 



...

Accuracy: 19.93 - Loss: 0.015378759968641848 



100%|██████████| 31/31 [00:10<00:00,  2.82it/s]


Accuracy: 19.45 - Loss: 0.015439580614920578 



100%|██████████| 31/31 [00:10<00:00,  2.93it/s]


Accuracy: 19.37 - Loss: 0.015470881108929408 



#### 2.3.9 - Art Painting &rightarrow; Photo, mor = 0.75

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': '2_3_9',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='2_3_9', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
layer_name layer1.0.conv1 random_mask sum():  tensor(50272., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.75
layer_name layer1.1.conv2 random_mask sum():  tensor(50075., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.75
layer_name layer2.1.conv1 random_mask sum():  tensor(25010.

100%|██████████| 14/14 [00:05<00:00,  2.37it/s]


Accuracy: 24.19 - Loss: 0.015956015215662426 



100%|██████████| 14/14 [00:05<00:00,  2.45it/s]


Accuracy: 26.05 - Loss: 0.015752324087177208 



...

100%|██████████| 14/14 [00:04<00:00,  3.31it/s]


Accuracy: 25.57 - Loss: 0.015503694911202985 



100%|██████████| 14/14 [00:04<00:00,  3.20it/s]


Accuracy: 27.78 - Loss: 0.015376295443780408 



 31%|███▏      | 5/16 [00:04<00:05,  2.02it/s]

## 3 - Adapting Activation Maps across Domains

### 3.1 - module placement: layer2.1.conv2

#### 3.1.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': '3_1_1',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 3.1.2 - Art Painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': '3_1_2',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 3.1.3 - Art Painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Art Painting -> Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': '3_1_3',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

## Ext. 2 - Binarization Ablation